##Task 2: Steam Game Recommendation System using Spark AL

 Project Overview: This project develops a Collaborative Filtering Recommender System leveraging Apache Spark's ALS (Alternating Least Squares) algorithm. The dataset used is Steam's user play history, which includes:

- User IDs

- Game titles

- Behavior (either purchase or play)

- Hours played or purchase indicator

The objective is to recommend games to users based on their behavior in the system.

###1.0 Importing Libraries

Importing Necessary Libraries for the Recommendation Pipeline: In this code, we import several crucial libraries and modules to construct and assess a recommendation system using PySpark and MLflow:

- mlflow and mlflow.spark: These are used for tracking machine learning experiments, logging models, parameters, and metrics. Specifically, mlflow.spark allows logging and loading of Spark ML models.

- pyspark.sql.functions (such as col, count, desc): These functions are used for manipulating data, including selecting columns, counting values, and sorting data in descending order.

- StringIndexer: This feature transformer converts categorical string columns (e.g., user or item IDs) into numerical indices, which are necessary for machine learning models.

- ALS (Alternating Least Squares): This collaborative filtering algorithm is utilized to build the recommendation model based on interactions between users and items.

- RegressionEvaluator: This is used to evaluate the model's performance using metrics like RMSE (Root Mean Squared Error).

- ParamGridBuilder: This tool helps define a grid of hyperparameters for tuning the ALS model.

- TrainValidationSplit and CrossValidator: These are tools for hyperparameter tuning. TrainValidationSplit divides the dataset into training and validation sets, while CrossValidator performs k-fold cross-validation to identify the best model parameters.

In [0]:
import mlflow
import mlflow.spark
from pyspark.sql.functions import col, count, desc
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.tuning import CrossValidator


###1.1 MLflow AutoLogging


The command mlflow.pyspark.ml.autolog() activates automatic logging for PySpark ML models. It enables tracking of parameters, metrics, and models without the need for additional code. This feature simplifies the process of managing and comparing various model runs in MLflow, helping to save time and maintain a well-organized record of experiments.

In [0]:
mlflow.pyspark.ml.autolog()


The command dbutils.fs.ls("/FileStore/tables/") shows the list of files and folders stored in the /FileStore/tables/ directory in Databricks. It's useful to check which files are available for use in my notebook.

In [0]:
dbutils.fs.ls("/FileStore/tables/")

[FileInfo(path='dbfs:/FileStore/tables/Clinicaltrial_16012025.csv', name='Clinicaltrial_16012025.csv', size=205522181, modificationTime=1742335744000),
 FileInfo(path='dbfs:/FileStore/tables/Occupancy_Detection_Data.csv', name='Occupancy_Detection_Data.csv', size=50968, modificationTime=1740590400000),
 FileInfo(path='dbfs:/FileStore/tables/account-models/', name='account-models/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/accounts/', name='accounts/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations/', name='activations/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations.zip', name='activations.zip', size=8411369, modificationTime=1739632416000),
 FileInfo(path='dbfs:/FileStore/tables/emails.csv', name='emails.csv', size=1426122219, modificationTime=1739827992000),
 FileInfo(path='dbfs:/FileStore/tables/iotstream/', name='iotstream/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tab

###1.2 Load and Preprocess the Dataset


We start by importing the Steam 200k dataset into a Spark DataFrame. The dataset is located at /FileStore/tables/steam_200k.csv, and we utilize the read.csv() function from Spark to load it. As the dataset lacks column headers, we specify header=False, prompting Spark to assign default column names (_c0, _c1, _c2, _c3). Additionally, we enable inferSchema=True so that Spark can automatically determine the data types of each column. Lastly, we use .show(5) to display the first five rows, ensuring the dataset is loaded correctly.

In [0]:
df_steam = spark.read.csv('/FileStore/tables/steam_200k.csv', header=False, inferSchema=True)
df_steam.show(5)

+---------+--------------------+--------+-----+
|      _c0|                 _c1|     _c2|  _c3|
+---------+--------------------+--------+-----+
|151603712|The Elder Scrolls...|purchase|  1.0|
|151603712|The Elder Scrolls...|    play|273.0|
|151603712|           Fallout 4|purchase|  1.0|
|151603712|           Fallout 4|    play| 87.0|
|151603712|               Spore|purchase|  1.0|
+---------+--------------------+--------+-----+
only showing top 5 rows



###1.3 Renaming column Names

This code renames the columns of the df_steam DataFrame to provide more descriptive and user-friendly names. The default column names (_c0, _c1, _c2, _c3) are replaced with user_id, game, behavior, and value. This enhances readability and simplifies the understanding of the dataset. The show(5) function is used at the end to display the first five rows of the updated DataFrame, confirming that the column renaming has been successfully applied.

In [0]:
# Rename columns
df_steam = df_steam.withColumnRenamed("_c0", "user_id") \
       .withColumnRenamed("_c1", "game") \
       .withColumnRenamed("_c2", "behavior") \
       .withColumnRenamed("_c3", "value")
df_steam.show(5)

+---------+--------------------+--------+-----+
|  user_id|                game|behavior|value|
+---------+--------------------+--------+-----+
|151603712|The Elder Scrolls...|purchase|  1.0|
|151603712|The Elder Scrolls...|    play|273.0|
|151603712|           Fallout 4|purchase|  1.0|
|151603712|           Fallout 4|    play| 87.0|
|151603712|               Spore|purchase|  1.0|
+---------+--------------------+--------+-----+
only showing top 5 rows



##2.0  EDA- Exploratory Data Analysis

###2.0.1 Displaying Behaviour types and their counts

This code performs a basic exploratory data analysis (EDA) task by counting the occurrences of each type of user behavior in the dataset. It groups the data based on the "behavior" column and uses the .count() function to calculate the number of records for each behavior type (such as play, purchase, etc.). The .show() function is then used to display the results in a table format. This step helps in understanding the distribution of user interactions with the games.

In [0]:
# Count of user behaviors
df_steam.groupBy("behavior").count().show()

+--------+------+
|behavior| count|
+--------+------+
|purchase|129511|
|    play| 70489|
+--------+------+



In [0]:
# Group by 'behavior' and count the occurrences
behavior_counts = df_steam.groupBy("behavior").count()

# Display as a pie chart in Databricks
display(behavior_counts)

behavior,count
purchase,129511
play,70489


Databricks visualization. Run in Databricks to view.

###2.0.2 Filtering the Dataset to display Only "Play" Behavior

This code filters the df_steam DataFrame to create a new DataFrame called play_df, which contains only the rows where the behavior column is equal to 'play'. The .show(5) function is then used to display the first 5 rows of the play_df DataFrame, providing a preview of the data where users have interacted with the games by playing them.

In [0]:
# Filter only 'play' events for implicit feedback
play_df = df_steam.filter(df_steam.behavior == 'play')
play_df.show(5)

+---------+--------------------+--------+-----+
|  user_id|                game|behavior|value|
+---------+--------------------+--------+-----+
|151603712|The Elder Scrolls...|    play|273.0|
|151603712|           Fallout 4|    play| 87.0|
|151603712|               Spore|    play| 14.9|
|151603712|   Fallout New Vegas|    play| 12.1|
|151603712|       Left 4 Dead 2|    play|  8.9|
+---------+--------------------+--------+-----+
only showing top 5 rows



###2.0.3 Top users by number of play

In this code, the play_df DataFrame is aggregated to count how many games each user has played. The data is first grouped by user_id, meaning the aggregation is performed for each unique user. The agg() function is used with count("game") to calculate the total number of games played by each user, which is then aliased as "play_count". After calculating the play counts, the orderBy(desc("play_count")) function sorts the users in descending order of their play counts, ensuring that users with the highest number of games played appear first. Finally, the .show() function displays the results, showing the user IDs and their respective play counts, helping to identify the most active users based on game play behavior.

In [0]:
# Top users by number of plays
play_df.groupBy("user_id").agg(count("game").alias("play_count"))\
       .orderBy(desc("play_count")).show()


+---------+----------+
|  user_id|play_count|
+---------+----------+
| 62990992|       498|
| 11403772|       314|
|138941587|       299|
| 47457723|       298|
| 49893565|       297|
| 24469287|       284|
| 48798067|       254|
| 36546868|       235|
| 51557405|       210|
| 17530772|       209|
|116876958|       208|
| 22301321|       207|
| 11373749|       204|
|   975449|       202|
| 33013552|       200|
| 38049880|       200|
| 53875128|       197|
| 10599862|       178|
| 26762388|       178|
|   298950|       175|
+---------+----------+
only showing top 20 rows



###2.0.4 Displaying Top 10 Played games

In this step, we identify the top 10 most played games in the dataset. This helps us understand which games are the most popular based on the number of unique users who have played them. The insights gained here can be useful for game recommendation systems, player engagement analysis, and trend identification.

In [0]:
# Top 10 most played games
top_10_games_df=play_df.groupBy("game").agg(count("user_id").alias("players"))\
       .orderBy(desc("players")).show(10)

+--------------------+-------+
|                game|players|
+--------------------+-------+
|              Dota 2|   4841|
|     Team Fortress 2|   2323|
|Counter-Strike Gl...|   1377|
|            Unturned|   1069|
|       Left 4 Dead 2|    801|
|Counter-Strike So...|    715|
|The Elder Scrolls...|    677|
|         Garry's Mod|    666|
|      Counter-Strike|    568|
|Sid Meier's Civil...|    554|
+--------------------+-------+
only showing top 10 rows



###2.0.5 Bar Chart for top 10 most played games

In [0]:
# Bar Chart: Top 10 most played games
top_games = play_df.groupBy("game").agg(count("user_id").alias("players"))\
    .orderBy(desc("players")).limit(10)
top_games.display()

game,players
Dota 2,4841
Team Fortress 2,2323
Counter-Strike Global Offensive,1377
Unturned,1069
Left 4 Dead 2,801
Counter-Strike Source,715
The Elder Scrolls V Skyrim,677
Garry's Mod,666
Counter-Strike,568
Sid Meier's Civilization V,554


Databricks visualization. Run in Databricks to view.

The display(df_steam) function in Databricks allows us to visually explore the dataset in an interactive table format. Unlike .show(), which only prints a limited number of rows, display() provides a more dynamic way to view, sort, filter, and visualize data

In [0]:
display(df_steam)

user_id,game,behavior,value
151603712,The Elder Scrolls V Skyrim,purchase,1.0
151603712,The Elder Scrolls V Skyrim,play,273.0
151603712,Fallout 4,purchase,1.0
151603712,Fallout 4,play,87.0
151603712,Spore,purchase,1.0
151603712,Spore,play,14.9
151603712,Fallout New Vegas,purchase,1.0
151603712,Fallout New Vegas,play,12.1
151603712,Left 4 Dead 2,purchase,1.0
151603712,Left 4 Dead 2,play,8.9


This code removes unnecessary columns from the DataFrame play_df and creates a new DataFrame df with the remaining columns.

In [0]:
df = play_df.drop("confidence", "game_id2", "game_id_new", "log_transformed_value1")
df.show()

+---------+--------------------+--------+-----+
|  user_id|                game|behavior|value|
+---------+--------------------+--------+-----+
|151603712|The Elder Scrolls...|    play|273.0|
|151603712|           Fallout 4|    play| 87.0|
|151603712|               Spore|    play| 14.9|
|151603712|   Fallout New Vegas|    play| 12.1|
|151603712|       Left 4 Dead 2|    play|  8.9|
|151603712|            HuniePop|    play|  8.5|
|151603712|       Path of Exile|    play|  8.1|
|151603712|         Poly Bridge|    play|  7.5|
|151603712|         Left 4 Dead|    play|  3.3|
|151603712|     Team Fortress 2|    play|  2.8|
|151603712|         Tomb Raider|    play|  2.5|
|151603712|     The Banner Saga|    play|  2.0|
|151603712|Dead Island Epidemic|    play|  1.4|
|151603712|   BioShock Infinite|    play|  1.3|
|151603712|Dragon Age Origin...|    play|  1.3|
|151603712|Fallout 3 - Game ...|    play|  0.8|
|151603712|SEGA Genesis & Me...|    play|  0.8|
|151603712| Grand Theft Auto IV|    play

###3.0 Indexing the 'game' Column into Integer IDs


Since the game column contains string values (e.g., "Dota 2", "PUBG"), we need to convert it into numerical values before using it in a machine learning model like ALS (Alternating Least Squares). The StringIndexer function assigns a unique integer ID to each game

We are using the StringIndexer to convert the categorical "game" column, which contains game names, into numeric values. Since machine learning models require numerical input, converting categorical variables into numbers is an essential preprocessing step. The StringIndexer automatically assigns a unique integer index to each game in the dataset, creating a new column called "game_id_new" to store these integer values. This transformation makes the data compatible for use with machine learning models. After applying the indexing, we use printSchema() to confirm that the new "game_id_new" column has been successfully added as an integer type, ensuring the transformation is correct and the data is ready for further modeling.










In [0]:

game_indexer1 = StringIndexer(inputCol="game", outputCol="game_id_new")


df_steam = game_indexer1.fit(df_steam).transform(df_steam)


df_steam.printSchema()  


2025/04/17 00:47:32 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '699781f6d88247c99a5453a2ca1bc477', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow


root
 |-- user_id: integer (nullable = true)
 |-- game: string (nullable = true)
 |-- behavior: string (nullable = true)
 |-- value: double (nullable = true)
 |-- game_id_new: double (nullable = false)



###3.1 Applying Log Transformation to Data in PySpark

In data preprocessing, applying a log transformation is a widely used technique, particularly for addressing skewed data distributions. The given PySpark code illustrates how to apply this transformation to the "value" column of the df_steam DataFrame. The log transformation helps normalize the data by stabilizing variance and minimizing the influence of extreme values, which is beneficial for machine learning algorithms and statistical analyses that perform better with normally distributed data.

The process starts by importing necessary functions from PySpark: log1p, round, and col. The log1p function calculates the natural logarithm of 1 + x for each value in the column. This is a safer method for computing logarithms when the data might include zeros or negative numbers. Unlike the standard logarithm (log(x)), which is undefined for zero or negative values, log1p(x) adds 1 to the value before applying the logarithm, allowing the transformation to be applied more broadly without errors.

After performing the log transformation, the round() function is used to round the resulting values to two decimal places. This rounding step is crucial for cleaning up the transformed values and making them easier to interpret, particularly when dealing with large datasets. Rounding ensures the transformed values are concise and ready for further analysis or visualization.

The withColumn() function is then used to add a new column to the DataFrame, named "log_transformed_value", which holds the log-transformed and rounded values. This new column allows for easy comparison with the original data. Finally, the df_steam.show() function is used to display the first few rows of the updated DataFrame, enabling the user to verify the results of the transformation.

Overall, this technique is especially helpful when the data is highly skewed or contains outliers, as it reduces the impact of extreme values and prepares the dataset for more effective modeling and analysis. Log transformations allow analysts to work with data that exhibits a more normalized distribution, ultimately enhancing the performance of machine learning models and statistical methods.

In [0]:
from pyspark.sql.functions import log1p, round, col

df_steam = df_steam.withColumn(
    "log_transformed_value", 
    round(log1p(col("value")), 2)  # Round off after the log transformation
)
df_steam.show()


+---------+--------------------+--------+-----+-----------+---------------------+
|  user_id|                game|behavior|value|game_id_new|log_transformed_value|
+---------+--------------------+--------+-----+-----------+---------------------+
|151603712|The Elder Scrolls...|purchase|  1.0|        8.0|                 0.69|
|151603712|The Elder Scrolls...|    play|273.0|        8.0|                 5.61|
|151603712|           Fallout 4|purchase|  1.0|      100.0|                 0.69|
|151603712|           Fallout 4|    play| 87.0|      100.0|                 4.48|
|151603712|               Spore|purchase|  1.0|      332.0|                 0.69|
|151603712|               Spore|    play| 14.9|      332.0|                 2.77|
|151603712|   Fallout New Vegas|purchase|  1.0|       29.0|                 0.69|
|151603712|   Fallout New Vegas|    play| 12.1|       29.0|                 2.57|
|151603712|       Left 4 Dead 2|purchase|  1.0|        4.0|                 0.69|
|151603712|     

###3.2 Splitting Data into Training and Testing Sets in PySpark

The code shows how to split the df_steam DataFrame into two parts: one for training the model (train_df) and one for testing it (test_df). It uses randomSplit([0.8, 0.2], seed=100), which means 80% of the data goes to training and 20% to testing.

The seed=100 part makes sure the split happens the same way every time you run the code, which helps keep things consistent when you're building and testing your machine learning model.

In [0]:
train_df, test_df = df_steam.randomSplit([0.8, 0.2], seed=100)

###4.0 ALS Model Setup for Collaborative Filtering in PySpark


This code sets up an ALS (Alternating Least Squares) model using PySpark to build a recommendation system based on user behavior, like game plays or purchases. ALS works well when we don’t have explicit ratings—just actions like how long someone played a game.

Explanation of Parameters:

- maxIter=5: The model will run 5 times to improve accuracy.

- regParam=0.1: Helps prevent overfitting by adding a small penalty for complexity.

rank=50: Uses 50 hidden features to learn user-game patterns.

- alpha=alpha: Adjusts how much trust we put in user actions like plays or purchases.

- userCol="user_id": Column that identifies each user.

- itemCol="game_id_new": Column that identifies each game (converted to numeric).

- ratingCol="log_transformed_value": Uses a transformed version of user interaction (e.g., playtime) for training.

- implicitPrefs=True: Tells the model we’re using implicit data (like playing a game, not giving a star rating).

- coldStartStrategy="drop": Removes predictions for users or items the model hasn't seen before to avoid errors.

In short, this setup lets the model recommend games based on how users interact with other games, even when there are no star ratings—just behavior data.

In [0]:
als = ALS(
    maxIter=5, 
    regParam=0.1,  # Regularization parameter
    rank=50,       # Latent factors (rank)
    alpha=5,   # Confidence scaling for implicit feedback
    userCol="user_id",  # Use the existing user_id column (already integer)
    itemCol="game_id_new",  # Use indexed game (game_id)
    ratingCol="log_transformed_value",  # Using the log-transformed value as the rating
    implicitPrefs=True,  # Enable implicit feedback (purchase or play behavior)
    coldStartStrategy="drop"  # Drop rows with NaN predictions due to cold start
)

#### 4.0.1 Fitting the ALS Model on Training Data


In this line of code, the Alternating Least Squares (ALS) model is being fitted to the training data (train_df). The fit() method is used to train the model based on the provided training DataFrame. Here's a breakdown of what happens:

- als.fit(train_df): This method trains the ALS model using the training dataset (train_df) created earlier. The model learns the patterns between users and items (games) based on the log-transformed values (log_transformed_value column).

Once the model is fitted, it will have learned the latent factors that explain the relationship between users and items, and it can then be used to make predictions or recommendations based on unseen data.

In [0]:
# Fit the ALS model on the training data
model = als.fit(train_df)

2025/04/17 00:47:39 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '7fe203318e5448d893e07aff1f5359e4', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2025/04/17 00:47:40 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/databricks/python/lib/python3.10/site-packages/mlflow/data/spark_dataset.py:159: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missi

####4.0.2 Making Predictions Using the ALS Model on Test Data


In this step, the trained ALS model is used to make predictions on the test data:

model.transform(test_df): This applies the trained ALS model to test_df, predicting how strongly each user might interact with each game (based on their past behavior).

The result is a new DataFrame called predictions that includes user IDs, game IDs, and the predicted values. These predictions can later be compared with actual data to evaluate how well the model performs using metrics like RMSE or MAE.

In [0]:
# Make predictions on the test data
predictions = model.transform(test_df)

###5.0 Defining the Parameter Grid for Hyperparameter Tuning


This code sets up a list of different options to test for the ALS model's settings (hyperparameters) to see which ones work best:

ParamGridBuilder() starts building the list of options.

- .addGrid(als.rank, [10, 20]): Tests the model with 10 or 20 hidden factors (which help explain user-game preferences).

- .addGrid(als.maxIter, [8, 10]): Tries running the model for 8 or 10 iterations to see which works better.

- .addGrid(als.regParam, [0.05, 0.2]): Tests regularization values of 0.05 and 0.2, which helps prevent the model from overfitting.

- .build() combines all these options into a grid that will be used to find the best settings for the model.

The idea is to find the best combination of settings that gives the best performance when training the ALS model.

In [0]:
#  Param grid
param_grid = ParamGridBuilder() \
    .addGrid(als.rank, [10, 20]) \
    .addGrid(als.maxIter, [8, 10]) \
    .addGrid(als.regParam, [0.05, 0.2]) \
    .build()

###6.0 Evaluating Model Performance Using RMSE


Evaluating Model Performance Using RMSE
In this code, a RegressionEvaluator is used to evaluate the performance of the ALS model using the Root Mean Squared Error (RMSE) metric. Here's a breakdown of the process:

- RegressionEvaluator(): This class is used to evaluate the performance of a regression model. It requires the following parameters:

- metricName="rmse": Specifies that the evaluation metric to use is RMSE. RMSE is commonly used to measure the accuracy of continuous prediction models, where lower values indicate better performance.

- labelCol="log_transformed_value": Specifies the column containing the true values (actual ratings or behaviors). In this case, it is the "log_transformed_value" column from the test data.

- predictionCol="prediction": Specifies the column containing the predicted values. This would be the column generated by the ALS model when making predictions.

- evaluator.evaluate(predictions): The evaluate() method computes the RMSE between the actual and predicted values in the predictions DataFrame. It compares the "log_transformed_value" column (actual ratings) and the "prediction" column (predicted ratings).

- print(f"Root Mean Squared Error (RMSE): {rmse}"): Finally, the RMSE value is printed to the console. A lower RMSE indicates better model performance, as it reflects smaller errors in the model's predictions.

This evaluation step is essential for assessing how well the ALS model has learned to predict user-item interactions in the test set.

In [0]:
# Evaluate RMSE (Root Mean Squared Error)
evaluator = RegressionEvaluator(
    metricName="rmse", 
    labelCol="log_transformed_value", 
    predictionCol="prediction"
)
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 1.331557143268122


####6.0.1 Selecting Top Predictions for Game Recommendations


This code generates the top 10 game recommendations based on predicted ratings from the ALS model. It first removes any duplicate game entries with .dropDuplicates(["game"]) to ensure each game appears only once. The games are then sorted by their predicted ratings in descending order using .orderBy(col("prediction").desc()), so the highest-rated games are prioritized. The relevant columns—user ID, game name, and predicted rating—are selected with .select("user_id", "game", "prediction"), focusing only on the necessary data. To narrow it down, .limit(10) selects the top 10 highest-rated games. Finally, .show(truncate=False) displays the recommendations without truncating any long values, presenting a clean list of the most recommended games for users.

In [0]:
from pyspark.sql.functions import col

top_predictions = predictions \
    .dropDuplicates(["game"]) \
    .orderBy(col("prediction").desc()) \
    .select("user_id", "game", "prediction") \
    .limit(10)

top_predictions.show(truncate=False)


+--------+---------------------+----------+
|user_id |game                 |prediction|
+--------+---------------------+----------+
|32467994|Magic Duels          |1.1876781 |
|298950  |Hitman Absolution    |1.1698799 |
|975449  |BIT.TRIP RUNNER      |1.1365536 |
|7440594 |Path of Exile        |1.1356977 |
|8585433 |Monaco               |1.1337126 |
|30246419|Anomaly Warzone Earth|1.1258683 |
|298950  |Hotline Miami        |1.1125319 |
|8585433 |Risk of Rain         |1.1113329 |
|975449  |VVVVVV               |1.1096699 |
|298950  |Dishonored           |1.1013837 |
+--------+---------------------+----------+



##5.1 Hyperparameter Tuning

This code sets up hyperparameter tuning for the ALS model using TrainValidationSplit, which helps find the optimal combination of parameters more efficiently. Unlike cross-validation, which can be slow due to testing every possible combination, TrainValidationSplit divides the training data into two parts: 80% for training and 20% for validation (specified by trainRatio=0.8).

The estimator is set to the ALS model, and the param_grid defines the different values to be tested for parameters like rank, maxIter, and regParam. The evaluator is used to measure model performance, in this case, using RMSE (Root Mean Squared Error). This setup enables Spark to train the model multiple times with different parameter combinations and then select the best one based on the evaluation results.

In [0]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
# Use TrainValidationSplit for faster tuning (CrossValidator is slower)
tvs = TrainValidationSplit(
    estimator=als,
    estimatorParamMaps=param_grid,
    evaluator=evaluator,
    trainRatio=0.8
)


###5.1.1 Model Training and Selection with TrainValidationSplit

In this code, tvs.fit(train_df) trains the ALS model using the hyperparameter combinations defined earlier, performing both training and validation on the training dataset (train_df). After completing the training process, tvs_model.bestModel retrieves the version of the ALS model that achieved the best performance based on the evaluation metric (such as RMSE). This ensures that the model selected is the one with the most optimal parameters for making accurate predictions.

In [0]:
# Train the best model
tvs_model = tvs.fit(train_df)
best_model = tvs_model.bestModel

2025/04/17 00:48:36 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '433c5da2b4394ac1968109673a92c0c8', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2025/04/17 00:53:36 WARNING mlflow.pyspark.ml: Model TrainValidationSplitModel_3b097a05bf85 will not be autologged because it is not allowlisted or or because one or more of its nested models are not allowlisted. Call mlflow.spark.log_model() to explicitly log the model, or specify a custom allowlist via the spark.mlflow.pysparkml.autolog.logModelAllowlistFile Spark conf (see mlflow.pyspark.ml.autolog docs for more info).


#### Before applying hyperparameter tuning, the ALS model produced a relatively good RMSE score on the test data. However, after tuning the model using TrainValidationSplit and a defined parameter grid, the RMSE slightly increased. This may be due to the model becoming more generalized to avoid overfitting, or limitations in the validation split and parameter search space. While tuning is meant to improve performance, in some cases, the untuned model may incidentally align better with the specific test data used.

###5.1.2 Evaluating RMSE after Hyperparameter Tunning

This code uses the best-performing ALS model, obtained through hyperparameter tuning, to make predictions on the test dataset (test_df) using best_model.transform(test_df). The predictions are then evaluated using the RegressionEvaluator, which computes the Root Mean Squared Error (RMSE) between the predicted ratings and the actual values in the test set. Finally, the RMSE value is printed with print(f"Best Model RMSE: {rmse}"), providing a measure of how well the model performs on unseen data. Lower RMSE values indicate better model performance.

In [0]:
# Evaluate the best model
predictions = best_model.transform(test_df)
rmse = evaluator.evaluate(predictions)
print(f"Best Model RMSE: {rmse}")

Best Model RMSE: 1.334833728487273


###5.1.3 Logging the Best Hyperparameters

This code logs the best hyperparameters used in the selected ALS model. By accessing the underlying Java object of the best model (best_model._java_obj.parent()), it retrieves the values of the hyperparameters that performed the best during training and validation. The specific parameters logged are:

- rank: The number of latent factors (hidden features) learned by the model.

- regParam: The regularization parameter, which helps prevent overfitting.

- alpha: The confidence scaling parameter for implicit feedback (such as play or purchase).

- maxIter: The maximum number of iterations the ALS algorithm will run during training.

These values are printed to the console to track and understand the configuration of the optimized model.

In [0]:
# Log best hyperparameters
print(f"Best rank: {best_model._java_obj.parent().getRank()}")
print(f"Best regParam: {best_model._java_obj.parent().getRegParam()}")
print(f"Best alpha: {best_model._java_obj.parent().getAlpha()}")
print(f"Best maxIter: {best_model._java_obj.parent().getMaxIter()}")

Best rank: 20
Best regParam: 0.05
Best alpha: 5.0
Best maxIter: 10


###7.0 Generating Top Game Recommendations

In this code, best_model.recommendForAllUsers(5) generates the top 5 game recommendations for each user based on the trained ALS model. The method uses the best model to predict which games each user is likely to enjoy, and it returns a list of these top recommendations for all users in the dataset. The .show(truncate=False) function displays the results, showing the user ID along with the recommended games and their corresponding predicted ratings. This helps generate personalized recommendations for users based on their past behavior and the patterns learned by the ALS model.

In [0]:
recommendations = best_model.recommendForAllUsers(5)
recommendations.show(truncate=False)


+--------+-----------------------------------------------------------------------------------------+
|user_id |recommendations                                                                          |
+--------+-----------------------------------------------------------------------------------------+
|76767   |[{27, 1.0841709}, {25, 1.0553992}, {6, 1.0545417}, {23, 0.9348383}, {50, 0.9271765}]     |
|144736  |[{6, 1.0205216}, {19, 0.9098968}, {28, 0.90006214}, {23, 0.8876451}, {34, 0.86714137}]   |
|229911  |[{10, 1.1543086}, {13, 1.1368946}, {6, 1.113492}, {28, 1.0751214}, {5, 1.0592492}]       |
|835015  |[{6, 0.89783055}, {28, 0.8181273}, {19, 0.7930749}, {34, 0.7878311}, {23, 0.77616256}]   |
|948368  |[{6, 1.0288084}, {5, 0.9886271}, {13, 0.9647761}, {10, 0.9568608}, {19, 0.91974235}]     |
|975449  |[{177, 1.203975}, {32, 1.1765326}, {340, 1.122456}, {157, 1.1194494}, {49, 1.1153624}]   |
|1268792 |[{6, 0.9120937}, {19, 0.8117956}, {28, 0.80493265}, {23, 0.79238737}, {34, 0.7759

###7.1  Extracting and Displaying Top Recommended Games from ALS Model

This code processes the ALS model's game recommendations by first exploding the nested recommendation arrays into individual rows for each user. It then extracts the game ID and predicted score from each recommendation and groups the data by game ID to count how often each game appears in the recommendations. The games are sorted by this count in descending order to highlight the most frequently recommended games. Finally, it displays the top 10 most recommended games, providing insights into which games are most popular across all users.

In [0]:
from pyspark.sql.functions import explode, col


exploded_recs = recommendations_df.select(
    col("user_id"),
    explode("recommendations").alias("rec")
)

exploded_recs = exploded_recs.select(
    col("user_id"),
    col("rec").getItem("game_id_new").alias("game_id"),
    col("rec").getItem("rating").alias("score")
)

top_recommended_games = exploded_recs.groupBy("game_id") \
    .count() \
    .orderBy(col("count").desc())

top_recommended_games.limit(10).display()


game_id,count
0,4287
9,4042
95,3268
123,3128
1,2953
248,2826
3,2736
2,2206
12,2034
31,1387


###8.0 Conclusion

In this task, a Collaborative Filtering Recommender System was developed using the Alternating Least Squares (ALS) algorithm on the Steam dataset to recommend games based on user behavior. The initial model, evaluated using Root Mean Squared Error (RMSE), achieved an RMSE of 1.33.

To optimize the model, hyperparameter tuning was conducted using TrainValidationSplit, which allowed for the efficient search of the best hyperparameter combinations. The best parameters identified were rank = 20, regParam = 0.05, alpha = 5.0, and maxIter = 10. After tuning, the RMSE slightly increased to 1.34, indicating a minor decrease in performance, but the hyperparameters helped improve the model's overall robustness.

The final model was used to generate personalized game recommendations for users. These results highlight the importance of hyperparameter tuning for improving the performance and predictive power of machine learning models, even if the impact on RMSE is marginal.